# 1. 영화 차트 가져오기

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do")

In [ ]:
from selenium.webdriver.common.by import By

search_m = driver.find_element(By.ID, "sSearchYearFrom")
search_year_list = search_m.find_elements(By.TAG_NAME, "option")

for idx, y_list  in enumerate(search_year_list):
    print(str(idx)+", "+y_list.get_attribute("value"))

#### 1-1. year 리스트 뽑아주기

In [ ]:
year_list=[y_list.get_attribute("value") for y_list in search_year_list]
year_list

In [ ]:
#필요한 데이터만 자르기 (2023-2014)
year_list = year_list[10:]
year_list.remove('2024')
print(len(year_list),year_list)

In [ ]:
#조회 버튼
driver.find_element(By.CLASS_NAME,"btn_blue").click()

In [ ]:
#확인 
import time
from tqdm.notebook import tqdm

for y_list in tqdm(year_list):
    search_m = driver.find_element(By.ID, "sSearchYearFrom")
    search_m.click()
    search_m.send_keys(y_list)
    driver.find_element(By.CLASS_NAME,"btn_blue").click()
    driver.implicitly_wait(3)

#### 1-2.  값 뽑아오기 

In [ ]:
result = driver.find_element(By.CLASS_NAME,"tbl_comm.th_sort")
movie = result.find_element(By.ID,"td_movie")

rnk = result.find_element(By.ID, "td_rank")
name = movie.find_element(By.CLASS_NAME, "ellip.per90")
date = result.find_element(By.ID, "td_openDt")
sale = result.find_element(By.ID,"td_salesAcc")
audience = result.find_element(By.ID, "td_audiAcc")
screen = result.find_element(By.ID, "td_scrnCnt")
screen_number = result.find_element(By.ID, "td_showCnt")

In [ ]:
result = driver.find_element(By.CLASS_NAME,"tbl_comm.th_sort")
rows = result.find_elements(By.TAG_NAME,"tr")[1:]

In [ ]:
import re 
from tqdm.notebook import tqdm

movie_chart_data=[]

for y_list in tqdm(year_list):
    search_m = driver.find_element(By.ID, "sSearchYearFrom")
    search_m.click()
    search_m.send_keys(y_list)
    driver.find_element(By.CLASS_NAME,"btn_blue").click()
    driver.implicitly_wait(3)

    result = driver.find_element(By.CLASS_NAME,"tbl_comm.th_sort")
    table = result.find_element(By.TAG_NAME,"tbody")
    rows = table.find_elements(By.TAG_NAME,"tr")


    for row in rows:
        movie = row.find_element(By.ID,"td_movie")
        rnk = row.find_element(By.ID, "td_rank").text
        name = row.find_element(By.CLASS_NAME, "ellip.per90").text
        date = row.find_element(By.ID, "td_openDt").text
        sale = row.find_element(By.ID,"td_salesAcc").text
        audience = row.find_element(By.ID, "td_audiAcc").text
        screen = row.find_element(By.ID, "td_scrnCnt").text
        screen_number = row.find_element(By.ID, "td_showCnt").text
        
        movie_chart = {
            "rank": rnk,
            "name": name,
            "date" : date,
            "sale" : sale,
            "audience" : audience,
            "screen" : screen,
            "screen_number" : screen_number
        }
        
        movie_chart_data.append(movie_chart)
        driver.implicitly_wait(1)

print(len(movie_chart_data), movie_chart_data[3:])

In [ ]:
driver.quit()

#### 1-3. pandas 저장 

In [ ]:
import pandas as pd 
df = pd.DataFrame(movie_chart_data)
df.head(5)

In [ ]:
df.to_csv("movie_chart_all.csv", index=False, encoding="utf-8-sig")

# 2. 넷플릭스 top 20 크롤링

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()

prefs = { 'download.default_directory':'/home/lim/dev_ws/eda/data',
         'download.prompt_for_download':False }
options.add_experimental_option('prefs', prefs)

url = "https://m.kinolights.com/ranking/netflix"
driver = webdriver.Chrome(service=Service("../driver/chromedriver"),
                          options = options)

driver.get(url)

!. 이때, 팝업창은 닫아주자. 

#### 2-1. 크롤링

In [9]:
table = driver.find_element(By.CLASS_NAME,"ranking-swiper-body__slide.swiper-slide.swiper-slide-active")
find_rank= table.find_element(By.TAG_NAME, "ul")
rows = find_rank.find_elements(By.TAG_NAME,"li")

In [10]:
# 테이블과 행 가져오기
table = driver.find_element(By.CLASS_NAME, "ranking-swiper-body__slide.swiper-slide.swiper-slide-active")
find_rank = table.find_element(By.TAG_NAME, "ul")
rows = find_rank.find_elements(By.CLASS_NAME, "ranking-item.extra")

netflix = []

# 랭크는 후처리 과정에서 인덱스 + 1 해서 추가
for i in range (len(rows)):
    table = driver.find_element(By.CLASS_NAME, "ranking-swiper-body__slide.swiper-slide.swiper-slide-active")
    find_rank = table.find_element(By.TAG_NAME, "ul")
    rows = find_rank.find_elements(By.CLASS_NAME, "ranking-item.extra")
    row=rows[i] 

    name = row.find_element(By.CLASS_NAME, "info__title").text
    score = row.find_element(By.CLASS_NAME, "score__number").text

    link_element = row.find_element(By.ID, "contentDailyRankingList")
    link_element.click()
    driver.implicitly_wait(3)
    

    metadata_items = driver.find_elements(By.CLASS_NAME, "metadata__item")
    genre, channel = "N/A", "N/A"
        
    for item in metadata_items:
        title = item.find_element(By.CLASS_NAME, "item__title").text.strip()
        body = item.find_element(By.CLASS_NAME, "item__body").text.strip()
            
        if title == "장르":
            genre = body
        elif title == "채널":
            channel = body


    netflix_data = {
        "name": name,
        "production": channel, 
        "genre": genre,
        "score": score,
    }
    netflix.append(netflix_data)
    

    driver.back()
    driver.implicitly_wait(1)


print(len(netflix), netflix[:3])


20 [{'name': '오징어 게임 시즌 2', 'production': '넷플릭스', 'genre': '드라마, 스릴러, 서스펜스, 공포(호러)', 'score': '78.6'}, {'name': '옥씨부인전', 'production': 'JTBC', 'genre': '드라마', 'score': '83.6'}, {'name': '지금 거신 전화는', 'production': 'MBC', 'genre': '멜로/로맨스, 스릴러', 'score': '85.9'}]


#### 2-2. pandas로 저장

In [11]:
import pandas as pd 
netflix = pd.DataFrame(netflix)
netflix

,name,production,genre,score
0,오징어 게임 시즌 2,넷플릭스,"드라마, 스릴러, 서스펜스, 공포(호러)",78.6
1,옥씨부인전,JTBC,드라마,83.6
2,지금 거신 전화는,MBC,"멜로/로맨스, 스릴러",85.9
3,체크인 한양,채널A,"드라마, 멜로/로맨스",80
4,나는 SOLO <나는 솔로>,"ENA, SBS, SBS PLUS",예능,79.4
5,퍼펙트 데이즈,N/A,드라마,98
6,파일럿,N/A,코미디,68.8
7,이혼숙려캠프: 새로고침,JTBC,예능,45.5
8,캐리온,N/A,"액션, 스릴러, 미스터리",90.7
9,베테랑 2,N/A,"액션, 범죄",78.8


In [12]:
driver.quit()

In [13]:
netflix.to_csv("netflix.csv", index=False, encoding="utf-8-sig")

# 2.티빙 top 20 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()

prefs = { 'download.default_directory':'/home/lim/dev_ws/eda/data',
         'download.prompt_for_download':False }
options.add_experimental_option('prefs', prefs)

url = "https://m.kinolights.com/ranking/tving"
driver = webdriver.Chrome(service=Service("../driver/chromedriver"),
                          options = options)

driver.get(url)

!. 팝업은 닫아주자

In [33]:
driver.get(url)

#### 3-1. 크롤링

In [2]:
table = driver.find_element(By.CLASS_NAME,"ranking-swiper-body__slide.swiper-slide.swiper-slide-active")
find_rank= table.find_element(By.TAG_NAME, "ul")
rows = find_rank.find_elements(By.TAG_NAME,"li")

In [3]:
# 테이블과 행 가져오기
table = driver.find_element(By.CLASS_NAME, "ranking-swiper-body__slide.swiper-slide.swiper-slide-active")
find_rank = table.find_element(By.TAG_NAME, "ul")
rows = find_rank.find_elements(By.CLASS_NAME, "ranking-item.extra")

tiving = []

# 랭크는 후처리 과정에서 인덱스 + 1 해서 추가
for i in range (len(rows)):
    table = driver.find_element(By.CLASS_NAME, "ranking-swiper-body__slide.swiper-slide.swiper-slide-active")
    find_rank = table.find_element(By.TAG_NAME, "ul")
    rows = find_rank.find_elements(By.CLASS_NAME, "ranking-item.extra")
    row=rows[i] 

    name = row.find_element(By.CLASS_NAME, "info__title").text
    score = row.find_element(By.CLASS_NAME, "score__number").text

    link_element = row.find_element(By.ID, "contentDailyRankingList")
    link_element.click()
    driver.implicitly_wait(3)
    

    metadata_items = driver.find_elements(By.CLASS_NAME, "metadata__item")
    genre, channel = "N/A", "N/A"
        
    for item in metadata_items:
        title = item.find_element(By.CLASS_NAME, "item__title").text.strip()
        body = item.find_element(By.CLASS_NAME, "item__body").text.strip()
            
        if title == "장르":
            genre = body
        elif title == "채널":
            channel = body


    tiving_data = {
        "name": name,
        "production": channel,  # 채널(배급사)
        "genre": genre,
        "score": score,
    }
    tiving.append(tiving_data)
    

    driver.back()
    driver.implicitly_wait(1)


print(len(tiving), tiving[:3])


20 [{'name': '옥씨부인전', 'production': 'JTBC', 'genre': '드라마', 'score': '83.6'}, {'name': '체크인 한양', 'production': '채널A', 'genre': '드라마, 멜로/로맨스', 'score': '80'}, {'name': '해리 포터와 마법사의 돌', 'production': 'N/A', 'genre': '판타지, 가족, 어드벤처(모험)', 'score': '96.8'}]


#### 3-2. pandas로 저장

In [5]:
import pandas as pd 
tiving = pd.DataFrame(tiving)
tiving

,name,production,genre,score
0,옥씨부인전,JTBC,드라마,83.6
1,체크인 한양,채널A,"드라마, 멜로/로맨스",80
2,해리 포터와 마법사의 돌,N/A,"판타지, 가족, 어드벤처(모험)",96.8
3,더 퍼스트 슬램덩크,N/A,"애니메이션, 드라마",97.6
4,나는 SOLO <나는 솔로>,"ENA, SBS, SBS PLUS",예능,79.4
5,해리 포터와 불의 잔,N/A,"판타지, 가족, 액션, 어드벤처(모험), 미스터리",94.5
6,이혼숙려캠프: 새로고침,JTBC,예능,45.5
7,존 오브 인터레스트,N/A,"드라마, 전쟁",95.4
8,더러운 돈에 손대지 마라,N/A,"범죄, 드라마",36
9,해리 포터와 비밀의 방,N/A,"판타지, 가족, 어드벤처(모험), 미스터리",96.8


In [6]:
driver.quit()

In [7]:
tiving.to_csv("tiving.csv", index=False, encoding="utf-8-sig")